In [1]:
import os 
%pwd

'/home/prathameshdevadiga/personal/projs/TextSummarization/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/prathameshdevadiga/personal/projs/TextSummarization'

In [4]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True) 
class DataTransformationConfig: 
    root_dir: Path 
    data_path: Path 
    tokenizer_name: Path 

In [5]:
from textSummarizer.constants import * 
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH): 
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_data_transformation_config(self) -> DataTransformationConfig: 
        config = self.config.data_transformation

        create_directories([config.root_dir]) 

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path, 
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [7]:
import os 
from textSummarizer.logging import logger 
from transformers import AutoTokenizer 
from datasets import load_dataset, load_from_disk

In [8]:
class DataTransformation: 
    def  __init__(self, config: DataTransformationConfig): 
        self.config = config 
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) 

    def convert_examples_to_features(self, example_batch): 
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True) 

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True) 
        
        return {
            'input_ids' : input_encodings['input_ids'], 
            'attention_mask': input_encodings['attention_mask'], 
            'labels': target_encodings['input_ids']
        }
    
    def convert(self): 
        dataset_samsum = load_from_disk(self.config.data_path) 
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True) 
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum-dataset"))


In [9]:
try: 
    config = ConfigurationManager() 
    data_transformation_config = config.get_data_transformation_config() 
    data_transformation = DataTransformation(config = data_transformation_config) 
    data_transformation.convert() 

except Exception as e: 
    raise e 

[2024-01-04 18:30:49,257 : INFO : common: yaml file : config/config.yaml is loaded successfully. ]
[2024-01-04 18:30:49,262 : INFO : common: yaml file : params.yaml is loaded successfully. ]
[2024-01-04 18:30:49,264 : INFO : common: creating directory at artifacts]
[2024-01-04 18:30:49,268 : INFO : common: creating directory at artifacts/data_transformation]


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]